In [1]:
# 所有的头

In [16]:
import torch # for save
import os
from van import Fan
from localconfig import config
import datetime
import random
from pprint import pprint
import html
from pyltp import SentenceSplitter
from pyltp import Postagger
from pyltp import Segmentor
from pyltp import NamedEntityRecognizer
import matplotlib.pyplot as plt
from tqdm import tqdm
from IPython.display import Image
from IPython.core.display import HTML 
import pyecharts
from pyecharts import Pie
from pyecharts import Line
%matplotlib inline

plt.rcParams['font.sans-serif']=['FangSong'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号


In [10]:
# 所有的配置

In [3]:
keyword = '北京大学' # 关注的关键词
until_year = 2000 # 最早的年份
until_num = 10000 # 截止的饭否数量
timeout = (60,60) # 超时时间

In [4]:
# 登录
fan = Fan(config['Consumer key'], config['Consumer secret'])
fan.xauth(config['my username'], config['my password'])

{'oauth_token': '1405515-071ad3666b5af9441d5cbb0dc1d3985b',
 'oauth_token_secret': '43cc293f5d127423f960b8636688577b'}

In [5]:
# 本小时内可用API次数
fan.request('GET', 'account/rate_limit_status', {}, timeout=timeout)

{'reset_time': 'Mon Mar 18 08:14:25 +0000 2019',
 'remaining_hits': 150,
 'hourly_limit': 150,
 'reset_time_in_seconds': 1552896865}

In [6]:
# 采集关键词消息
def datetimefromstr(s):
    return datetime.datetime.strptime(s, '%a %b %d %H:%M:%S %z %Y')
def get_status(keyword, until_year, until_num):
    latest = fan.request('GET', 'search/public_timeline', {'q':keyword, 'count':60}, timeout=timeout)
    while True:
        year = datetimefromstr(latest[-1]['created_at']).year
        if len(latest) >= until_num:
            break
        if year < until_year:
            break
        print(len(latest), year,'/', until_num, until_year)
        latest += fan.request('GET', 'search/public_timeline', { 'q':keyword, 'count':60, 'max_id':latest[-1]['id']}, timeout=timeout)[1:]
    return latest

In [7]:
# 下载并暂存
statuses = get_status(keyword, until_year, until_num)

60 2018 / 10000 2000
119 2018 / 10000 2000
178 2018 / 10000 2000
237 2017 / 10000 2000
296 2017 / 10000 2000
355 2017 / 10000 2000
414 2016 / 10000 2000
473 2016 / 10000 2000
532 2016 / 10000 2000
591 2015 / 10000 2000
650 2015 / 10000 2000
709 2015 / 10000 2000
768 2015 / 10000 2000
827 2015 / 10000 2000
886 2015 / 10000 2000
945 2015 / 10000 2000
1004 2015 / 10000 2000
1063 2015 / 10000 2000
1122 2015 / 10000 2000
1181 2015 / 10000 2000
1240 2014 / 10000 2000
1299 2014 / 10000 2000
1358 2014 / 10000 2000
1417 2014 / 10000 2000
1476 2013 / 10000 2000
1535 2013 / 10000 2000
1594 2013 / 10000 2000
1653 2013 / 10000 2000
1712 2013 / 10000 2000
1771 2013 / 10000 2000
1830 2013 / 10000 2000
1889 2013 / 10000 2000
1948 2013 / 10000 2000
2007 2013 / 10000 2000
2066 2013 / 10000 2000
2125 2013 / 10000 2000
2184 2013 / 10000 2000
2243 2013 / 10000 2000
2302 2013 / 10000 2000
2361 2013 / 10000 2000
2420 2013 / 10000 2000
2479 2013 / 10000 2000
2538 2013 / 10000 2000
2597 2013 / 10000 2000
2656 

In [8]:
torch.save(statuses,'tmp_statuses.torch.pkl')

In [9]:
# 恢复
# statuses = torch.load('tmp_statuses.torch.pkl')

In [11]:
print(len(statuses))

10031


In [60]:
# common
def get_top(l, num=5, reverse=False): # 从大到小
    cnt = {}
    for x in l:
        if type(x) == tuple and len(x) == 3:
            x = x[0]
        if x not in cnt:
            cnt[x] = 0
        cnt[x] += 1
    cnt_list = [(k,cnt[k],cnt[k]/len(l)) for k in cnt]
    cnt_list = sorted(cnt_list, key=lambda x:x[1])[::-1]
    if reverse:
        cnt_list = cnt_list[::-1]
    if num == -1:
        return cnt_list
    return cnt_list[:num]
def huabing(y): # x [(label,,0.xxx)]
    attr = [x[0] for x in y]
    v1 = [x[2] for x in y]
    if sum(v1) < 1.0:
        attr.append('其他')
        v1.append(1.0 - sum(v1))
    pie = Pie('')
    pie.add('', attr, v1, is_label_show=True)
    return pie

In [13]:
# 从消息关系本身入手

In [14]:
# 时间段分布

In [17]:
print(datetimefromstr(statuses[-1]['created_at']))
print(datetimefromstr(statuses[0]['created_at']))
begin_year = datetimefromstr(statuses[-1]['created_at']).year
end_year = datetimefromstr(statuses[0]['created_at']).year

2013-04-09 09:51:37+00:00
2019-03-11 06:35:26+00:00


In [18]:
huabing(get_top([datetimefromstr(x['created_at']).year for x in statuses], num = -1))

In [19]:
arr = sorted(get_top([datetimefromstr(x['created_at']).year for x in statuses],num=-1),key = lambda x:x[0])
attr = [str(x[0]) for x in arr ]
v1 = [x[1] for x in arr ]
line = Line('')
line.add('清华大学', attr, v1,is_smooth=True,is_label_show=True,mark_line=[ "average"])
# line.add("商家B", attr, v2, is_smooth=True, mark_line=["max", "average"])
line

In [22]:
# 通过sample，使得按年 normalize

In [23]:
# 用户信息文件
os.path.exists('./users.torch.pkl')

True

In [28]:
# 预处理用户信息
userid_to_info = {}
if os.path.exists('./users.torch.pkl'):
    userid_to_info = torch.load('users.torch.pkl')
for x in statuses:
    userid_to_info[x['user']['id']] = x['user']
torch.save(userid_to_info, 'users.torch.pkl')
print(len(userid_to_info))

3679


In [29]:
# 发表人数/总消息数量
uni_num = len(set([x['user']['id'] for x in statuses ]))
print(uni_num, len(statuses), uni_num/len(statuses))

837 10031 0.08344133187119929


In [30]:
# 发表top30的人
users_top = get_top([x['user']['id'] for x in statuses], num=30)
names_top = [ (userid_to_info[x[0]]['name'] , x[1], x[2]) for x in users_top]
pprint(names_top)
huabing(names_top)

[('胡萝卜素要好好生活', 1473, 0.14684478117834712),
 ('大包头很忙', 1443, 0.14385405243744392),
 ('Sloozy', 1344, 0.13398464759246337),
 ('落饭团', 741, 0.07387099990030904),
 ('zadbee', 580, 0.05782075565746187),
 ('pkugerald', 518, 0.051639916259595256),
 ('wocc', 496, 0.04944671518293291),
 ('henryxinjin', 439, 0.04376433057521683),
 ('星雲之外', 382, 0.038081945967500745),
 ('木子思想', 204, 0.02033695543814176),
 ('serenadeY', 158, 0.015751171368756853),
 ('罗妮zyzyyuan', 88, 0.008772804306649387),
 ('财经频道', 84, 0.00837404047452896),
 ('loseall', 71, 0.007078058020137574),
 ('大熊爸爸', 62, 0.006180839397866613),
 ('TraiPhumi', 62, 0.006180839397866613),
 ('Dryfish.Liu', 53, 0.005283620775595654),
 ('苏小东西', 41, 0.004087329279234373),
 ('壹杯', 36, 0.00358887448908384),
 ('猴三勿', 35, 0.0034891835310537334),
 ('抗心希古', 35, 0.0034891835310537334),
 ('atreides', 33, 0.0032898016149935203),
 ('阿cong', 31, 0.0030904196989333067),
 ('amyroyas', 29, 0.0028910377828730936),
 ('Yoyo_W', 23, 0.0022928920346924533),
 ('离愁的落叶',

In [31]:
def show_user_info(x):
    user = userid_to_info[x]
    pprint(user)
    return Image(url = user['profile_image_url_large'])

In [32]:
show_user_info(users_top[0][0])

{'birthday': '',
 'created_at': 'Sat Jan 12 06:20:30 +0000 2013',
 'description': '她不在这。',
 'favourites_count': 0,
 'followers_count': 6,
 'following': False,
 'friends_count': 26,
 'gender': '',
 'id': 'carotene',
 'location': '',
 'name': '胡萝卜素要好好生活',
 'notifications': False,
 'photo_count': 34,
 'profile_background_color': '#acdae5',
 'profile_background_image_url': 'http://static.fanfou.com/img/bg/0.png',
 'profile_background_tile': False,
 'profile_image_url': 'http://s3.meituan.net/v1/mss_3d027b52ec5a4d589e68050845611e68/avatar/s0/00/d8/k7.jpg?1357975181',
 'profile_image_url_large': 'http://s3.meituan.net/v1/mss_3d027b52ec5a4d589e68050845611e68/avatar/l0/00/d8/k7.jpg?1357975181',
 'profile_link_color': '#0066cc',
 'profile_sidebar_border_color': '#b2d1a3',
 'profile_sidebar_fill_color': '#e2f2da',
 'profile_text_color': '#222222',
 'protected': False,
 'screen_name': '胡萝卜素要好好生活',
 'statuses_count': 3031,
 'unique_id': '~xOeQhjYcIg0',
 'url': '',
 'utc_offset': 28800}


In [33]:
# 引用top10的消息
in_reply_to_top = get_top([x['in_reply_to_status_id'] for x in statuses], num=10)
pprint(in_reply_to_top)

[('', 10023, 0.9992024723357591),
 ('tbx52Lr9IKs', 1, 9.969095803010666e-05),
 ('aW74skcnctA', 1, 9.969095803010666e-05),
 ('FXEU4z75al8', 1, 9.969095803010666e-05),
 ('zsiBPeorD2U', 1, 9.969095803010666e-05),
 ('UKFCp5i0Edo', 1, 9.969095803010666e-05),
 ('ArHnBfKv5RY', 1, 9.969095803010666e-05),
 ('zNHsbeff4Q4', 1, 9.969095803010666e-05),
 ('RmbIhwUwA7E', 1, 9.969095803010666e-05)]


In [34]:
# 引用top10的人
in_reply_to_user_id_top = get_top([x['in_reply_to_user_id'] for x in statuses], num=10)
pprint(in_reply_to_user_id_top)

[('', 10022, 0.999102781377729),
 ('dearazure', 2, 0.00019938191606021333),
 ('maaaac', 1, 9.969095803010666e-05),
 ('Devil.cat', 1, 9.969095803010666e-05),
 ('tinynancy', 1, 9.969095803010666e-05),
 ('zfl001', 1, 9.969095803010666e-05),
 ('~R7Uiqg-_5VM', 1, 9.969095803010666e-05),
 ('dongdongqiang', 1, 9.969095803010666e-05),
 ('shuqingniandai', 1, 9.969095803010666e-05)]


In [35]:
# top20 gender
gender_top = get_top([x['user']['gender'] for x in statuses ], num=20)
pprint(gender_top)
huabing(gender_top)

[('', 6390, 0.6370252218123816),
 ('男', 2376, 0.23686571627953346),
 ('女', 1265, 0.12610906190808494)]


In [37]:
# top10 location
location_top10 = get_top
location_top10 = get_top([x['location'].split()[0] for x in statuses if len(x['location'].split())>=1], num=25)
pprint(location_top10)
huabing(location_top10)

[('39.989418,116.305854', 461, 0.04821671373287313),
 ('39.991835,116.310131', 357, 0.037339190461248825),
 ('39.989472,116.313103', 343, 0.03587490848237632),
 ('39.991884,116.309952', 333, 0.034828992783181675),
 ('39.991671,116.313012', 250, 0.02614789247986612),
 ('39.991877,116.317223', 207, 0.02165045497332915),
 ('39.987594,116.309135', 204, 0.021336680263570756),
 ('39.992138,116.315765', 203, 0.021232088693651292),
 ('39.991995,116.315247', 201, 0.021022905553812362),
 ('39.990801,116.314071', 190, 0.019872398284698254),
 ('北京', 190, 0.019872398284698254),
 ('39.988912,116.310372', 175, 0.018303524735906285),
 ('39.990131,116.315746', 172, 0.01798975002614789),
 ('39.987478,116.313326', 168, 0.017571383746470035),
 ('39.988740,116.311934', 161, 0.016839242757033784),
 ('39.987931,116.307391', 155, 0.016211693337516996),
 ('39.988385,116.310078', 155, 0.016211693337516996),
 ('39.988888,116.308947', 154, 0.016107101767597533),
 ('39.987770,116.305616', 153, 0.016002510197678066

In [38]:
def remove_atag(s):
    if '<a' not in s:
        return s
    start = s.find('>')
    end = s.find('</a>',start)
    return s[start+1:end]

In [39]:
# top20 source
source_top = get_top([remove_atag(x['source']) for x in statuses ], num=20)
pprint(source_top)
huabing(source_top)

[('街旁', 8893, 0.8865516897617386),
 ('网页', 289, 0.028810686870700826),
 ('享拍微博通', 152, 0.015153025620576214),
 ('FaWave', 116, 0.011564151131492374),
 ('饭否财经频道', 84, 0.00837404047452896),
 ('Android客户端', 73, 0.007277439936197787),
 ('「御飯 iPhone」', 63, 0.00628053035589672),
 ('御飯 iOS', 51, 0.00508423885953544),
 ('猫饭', 45, 0.0044860931113548),
 ('蘑菇饭', 42, 0.00418702023726448),
 ('手机上网', 35, 0.0034891835310537334),
 ('尚饭', 17, 0.0016947462865118134),
 ('饭否分享', 16, 0.0015950553284817066),
 ('PREFiX', 14, 0.0013956734124214933),
 ('Twidere', 13, 0.0012959824543913867),
 ('有饭', 11, 0.0010966005383311734),
 ('泡饭', 10, 0.0009969095803010668),
 ('Pinsta同步', 9, 0.00089721862227096),
 ('饭维基', 9, 0.00089721862227096),
 ('炒饭', 8, 0.0007975276642408533)]


In [40]:
# nlp part

In [41]:
def myfilter(s):
    s = s.replace('<strong>','')
    s = s.replace('</strong>','')
    while '转@' in s:
        pos = s.find('转@')
        end = len(s)
        for endsym in [' ', ']', '：',':','@']:
            end2 = s.find(endsym,pos+2)
            if end2 == -1:
                end2 = len(s)
            end = min(end,end2)
        if end-pos > 15:
            print('alert [转@] too long',s)
            s = s[:pos] + s[pos+2:]
        else:
            s = s[:pos] + s[end:]
    while '@' in s:
        pos = s.find('@')
        end = len(s)
        for endsym in [' ', ']', '：',':','@']:
            end2 = s.find(endsym,pos+1)
            if end2 == -1:
                end2 = len(s)
            end = min(end,end2)
        if end-pos > 15:
            print('alert [@] too long',s)
            s = s[:pos] + s[pos+1:]
        else:
            s = s[:pos] + s[end:]
    if '@' in s:
        print('alert @ in ', s)
    return s

In [42]:
sentences = []
for x in tqdm(statuses):
    s = html.unescape(x['text']) 
    s = myfilter(s)
    if keyword not in s:
        continue
    if '</' in s:
        print(s)
    ss = SentenceSplitter.split(s)
    for t in ss:
        if len(t) > 4:
            sentences.append(t)
    

 17%|█▋        | 1752/10031 [00:00<00:01, 6256.40it/s]

alert [@] too long @王頔很多考生来问我，664能不能去北大？我再次重申一下，664你可以考虑一下去清华，但是对于北大我只能说可以冲一下，但希望不大！毕竟至少696才能去北大一般都是坐四号线到北京大学东门站。坐696实在是有点堵，664只到清华西北门。不过如果你住在西直门的话375也行
alert [@] too long 【市场秩序的形成与审批制改革】北京大学教授 @zhangweiyingblog ：审批制、特许制给中国社会道德秩序带来很大伤害。要从市场秩序形成、法治社会建设角度去看待其弊端。没有审批制改革，就不可能有法治社会。 http://url.cn/PAOuMZ by FT中文网
alert [@] too long 我在北京大学图书馆南配殿。拖延症错过了大包头，完全可以过来看了大包头再回去洗澡的@
进门时遇小树和YO姐的逗比联盟，是这个名字么←_←
黄老师加油！ http://jie.pn/p_wACnYp0G
alert [@] too long 我在北京大学五四体育场。拖延症的后果之懒洋洋@
训练完后3个10的乐呵呵围观女篮新生杯聊八卦，后来加入了一个11的•﹏• http://jie.pn/p_wADeSJ0G
alert [@] too long 我在北京大学35楼。昨晚忘记闹闹钟了，但是今天八点要到技物楼做实验，然后6:50竟然醒了，闹了个7:05的闹钟，再醒的时候就7:15了@赞昳姐知道技物楼在哪，要不应该就迟到了→_→ http://jie.pn/p_wACCbJwG
alert [@] too long 我在北京大学图书馆教学参考书阅览室。弱弱的问一下今天的讲座大概讲到几点@....要不要把电脑扔在图书馆呢... http://jie.pn/p_wAAXk48G
alert [@] too long 我在北京大学家园食堂。预一下 11:40分牛肉和鱼@虽然带的不多【好吧其实是我昨天太饿了所以把其他的都吃完了。。。 http://jie.pn/p_wAAhAYoG
alert [@] too long 我在北京大学35楼。mor
不靠谱@小伙伴七点起来了，几点走的不清楚，自己八点才起来。。 http://jie.pn/p_wADd2HoG


 75%|███████▌  | 7549/10031 [00:00<00:00, 11111.40it/s]

alert [@] too long 我在北京大学45甲楼。一觉就睡到这个时候了@中午回来看到学妹们真是鲜嫩，赶紧换成短裤拖鞋，变成猥琐学长出门嗯 http://jie.pn/p_wAAnTmwG
alert [@] too long 我在北京大学康博思快餐西餐厅。原计划是五点半下班去打球的，都要错过brama姐姐和小明带来的好吃的了@结果一不小心上班上到了七点，就又可以去了•﹏• http://jie.pn/p_wADgVFgG
alert [@] too long 我在北京大学45甲楼。西班牙语课的时间好坑爹啊@....刚刚做了一件不好的事，深藏功与名~ http://jie.pn/p_IE3-BhMG
alert [@] too long 我在北京大学理教313。貌似成为了最后一个到教室的@旁边的考了一个小时多一点就交卷了，我还在那里就编结构。。 http://jie.pn/p_IE2oZQsG
alert [@] too long 我在北京大学理科教学楼：没带pad漫漫两节课怎么过@喂，这是你当助教的课啊……-_-||好吧 http://jie.pn/p_IE0yAf4F
alert [@] too long 我在北京大学燕南美食：目击怪阿姨新发型！感觉是按照头像剪得啊~。。眼前开始发黑了@老师我保证体力能再坚持考一个小时。。您就少出点题目吧TT http://jie.pn/p_IE2v9PwF
alert [@] too long 我在北京大学28楼：虽然大运转不行但是小运转也是别有风味…开始yy去宣化吃涮肉了@zhangyi…原因是看到了某人的某条优质攻略●﹏● http://jie.pn/p_IE3c1_gF
alert [@] too long 我在北京大学第二教学楼：呆呆在老板面前讲卟啉配合物@……我觉得老板可能听这个最认真了。。 http://jie.pn/p_IE0K7fcF
alert [@] too long 我在北京大学图书馆：@本校教师＆博士专用小自习室>3< http://jie.pn/p_Ic2eSPEF


100%|██████████| 10031/10031 [00:00<00:00, 15707.25it/s]

alert [@] too long 我在北京大学化学与分子工程学院：卧槽什么情况！！！注意安全呃@。。。别被ddl逼爆炸了。。[呼吁丁满来点个爱心？ http://jie.pn/p_IE1SZu8F
alert [@] too long 我在北京大学45甲楼426：跟起神打电话说晚上打球的事，结果黄老师跟我说了一通街旁周积分冲进前五之后就把电话给挂了@....既然如此的话，晚上9点到11点打羽毛球有谁要来么~配个双打轮着来~ http://jie.pn/p_IE3ocOAF
alert [@] too long 我在北京大学东门：本次会议主要对四大的筹划进行了简要讨论 并任命@gezideguaijiejie为组织委员 http://jie.pn/p_IE1KVtMF
alert [@] too long 我在北京大学35号楼：最近这是闹哪样啊@上上次洗澡，装东西的小框坏了。。昨天洗澡，戴的玉的绳子断了。。刚刚又把热水瓶给摔碎了。。 http://jie.pn/p_IE16Ms0F
alert [@] too long 我在北京大学三角地： @WilsonCheung我收到你们从大连发的明信片了，大连的戳是4月14号，什么水平。。。 http://jie.pn/p_IE1Zh8UF
alert [@] too long 我在北京大学三角地：路遇@maelranger以及一个道士 http://jie.pn/p_IE20g8QF
alert [@] too long 我在北京大学家园食堂：欢迎帝国金融专门人才 我们伟大的信科游泳队长 菠菜 同志 加入街旁。 发的第一条是‘在28楼签到，@李昱良，我来抢你地主了！’实际上，伦理委员会合并地点后我已经不是28楼地主了。。。欢迎大家点赞 http://jie.pn/p_IE32acIF
alert [@] too long 我在北京大学理教走廊自习区：@sloozy你是什么时候来的...一抬头就看到书没有了=。= http://jie.pn/p_IE1Au7sF


In [43]:
print(len(sentences))

19625


In [44]:
# 对sentences去重。。 拒绝转发的影响 （存疑？）

In [45]:
# sentences = list(set(sentences))

In [46]:
print(len(list(set(sentences))))

14582


In [47]:
# torch.save(['尼玛', '卧槽', '傻逼', '对了', '有木有', '他妈'],'blacklist.torch.pkl' )
def add_to_blacklist(w):
    global blacklist
    blacklist = torch.load('blacklist.torch.pkl')
    blacklist.append(w)
    torch.save(blacklist,'blacklist.torch.pkl')

In [48]:
# 无意义
blacklist = torch.load('blacklist.torch.pkl')

In [49]:
words_all = []

segmentor = Segmentor()
postagger = Postagger()
recognizer = NamedEntityRecognizer()
postagger.load('./ltp_data_v3.4.0/pos.model')
segmentor.load('./ltp_data_v3.4.0/cws.model')
recognizer.load('./ltp_data_v3.4.0/ner.model')

#todo 情感分析

for s in tqdm(sentences):
    words = segmentor.segment(s)
    postags = postagger.postag(words) #词性标注
    netags = recognizer.recognize(words, postags) # 命名实体识别
    for i in range(len(words)):
        if words[i] in blacklist:
            continue
        words_all.append((words[i],postags[i],netags[i]))

segmentor.release()
postagger.release()
recognizer.release()

100%|██████████| 14582/14582 [00:12<00:00, 1204.45it/s]


In [50]:
print(len(words_all))

150817


In [61]:
get_top([x[0] for x in words_all if len(x[0])>1 and x[1]!='wp'], num = 25)

[('北京大学', 5705, 0.07968544850127106),
 ('图书馆', 460, 0.00642511942341537),
 ('教学楼', 396, 0.005531189764505406),
 ('一个', 345, 0.004818839567561527),
 ('学院', 327, 0.0045674218509931),
 ('大学', 313, 0.004371874738106545),
 ('中国', 292, 0.004078554068776713),
 ('化学', 289, 0.004036651116015309),
 ('北大', 274, 0.0038271363522082856),
 ('医院', 259, 0.0036176215884012627),
 ('今天', 256, 0.003575718635639858),
 ('餐厅', 244, 0.00340810682459424),
 ('食堂', 232, 0.0032404950135486213),
 ('第二', 230, 0.003212559711707685),
 ('45', 230, 0.003212559711707685),
 ('没有', 222, 0.0031008185043439394),
 ('东门', 211, 0.002947174344218789),
 ('老师', 206, 0.0028773360896164482),
 ('吃饭', 195, 0.002723691929491298),
 ('还是', 190, 0.002653853674888957),
 ('一起', 189, 0.002639886023968489),
 ('理科', 188, 0.002625918373048021),
 ('第一', 184, 0.002570047769366148),
 ('北京', 173, 0.002416403609240998),
 ('农园', 169, 0.002360533005559125)]

In [53]:
posdict = {}
nedict = {}
for x in tqdm(words_all):
    if x[1] not in posdict:
        posdict[x[1]] = []
    if x[2] not in nedict:
        nedict[x[2]] = []
    posdict[x[1]].append(x)
    nedict[x[2]].append(x)

100%|██████████| 150817/150817 [00:00<00:00, 795204.36it/s]


In [54]:
# 词性
print(posdict.keys())


dict_keys(['v', 'u', 'r', 'wp', 'n', 'd', 'm', 'a', 'ns', 'c', 'q', 'nd', 'b', 'ws', 'nh', 'z', 'p', 'ni', 'nt', 'nl', 'j', 'nz', 'o', 'k', 'i', 'e', '%', 'h'])


In [62]:
# ['i'] 成语
pprint(get_top(posdict['i'], num=25))

[('吐槽', 25, 0.04807692307692308),
 ('丧心病狂', 12, 0.023076923076923078),
 ('好不容易', 9, 0.01730769230769231),
 ('差不多', 7, 0.013461538461538462),
 ('意识形态', 7, 0.013461538461538462),
 ('没什么', 6, 0.011538461538461539),
 ('绝大多数', 6, 0.011538461538461539),
 ('你好', 6, 0.011538461538461539),
 ('高等教育', 6, 0.011538461538461539),
 ('坑爹', 5, 0.009615384615384616),
 ('屌丝', 5, 0.009615384615384616),
 ('晚安', 5, 0.009615384615384616),
 ('事实上', 4, 0.007692307692307693),
 ('一不小心', 4, 0.007692307692307693),
 ('心满意足', 4, 0.007692307692307693),
 ('兼容并包', 4, 0.007692307692307693),
 ('您好', 3, 0.0057692307692307696),
 ('人山人海', 3, 0.0057692307692307696),
 ('社会科学', 3, 0.0057692307692307696),
 ('近年来', 3, 0.0057692307692307696),
 ('吃香锅', 3, 0.0057692307692307696),
 ('技术学校', 3, 0.0057692307692307696),
 ('高高在上', 3, 0.0057692307692307696),
 ('迫不及待', 3, 0.0057692307692307696),
 ('难以置信', 3, 0.0057692307692307696)]


In [63]:

# ['j'] 缩略词
pprint(get_top(posdict['j'], num=25))


[('北大', 274, 0.3281437125748503),
 ('高校', 59, 0.07065868263473053),
 ('清华', 51, 0.06107784431137724),
 ('二教', 51, 0.06107784431137724),
 ('三教', 50, 0.059880239520958084),
 ('人大', 11, 0.013173652694610778),
 ('十佳', 11, 0.013173652694610778),
 ('文博', 10, 0.011976047904191617),
 ('微博', 10, 0.011976047904191617),
 ('铁协', 10, 0.011976047904191617),
 ('美', 9, 0.010778443113772455),
 ('政法', 8, 0.009580838323353293),
 ('中', 8, 0.009580838323353293),
 ('社科', 7, 0.008383233532934131),
 ('环保', 6, 0.00718562874251497),
 ('中科院', 6, 0.00718562874251497),
 ('京', 5, 0.005988023952095809),
 ('财税', 5, 0.005988023952095809),
 ('大一', 5, 0.005988023952095809),
 ('国企', 5, 0.005988023952095809),
 ('文革', 5, 0.005988023952095809),
 ('一中', 5, 0.005988023952095809),
 ('房地产', 5, 0.005988023952095809),
 ('日', 5, 0.005988023952095809),
 ('联大', 4, 0.004790419161676647)]


In [64]:

# ['n'] 名词
pprint(get_top(posdict['n'], num=25))


[('楼', 699, 0.02750019671099221),
 ('人', 523, 0.020575969785191595),
 ('图书馆', 460, 0.018097411283342513),
 ('教学楼', 396, 0.015579510583051381),
 ('学院', 327, 0.012864898890550003),
 ('大学', 313, 0.012314108112361319),
 ('化学', 289, 0.011369895349752144),
 ('医院', 259, 0.010189629396490676),
 ('餐厅', 244, 0.009599496419859941),
 ('食堂', 232, 0.009127390038555354),
 ('老师', 206, 0.008104492879062081),
 ('理科', 188, 0.007396333307105201),
 ('农园', 169, 0.0066488315367062715),
 ('教授', 158, 0.0062160673538437326),
 ('时候', 150, 0.005901329766307342),
 ('区', 144, 0.005665276575655047),
 ('三角地', 123, 0.00483909040837202),
 ('家园', 123, 0.00483909040837202),
 ('分子', 121, 0.004760406011487922),
 ('中心', 117, 0.004603037217719726),
 ('工程学院', 114, 0.00448501062239358),
 ('阅览室', 112, 0.004406326225509481),
 ('学生', 112, 0.004406326225509481),
 ('地铁站', 104, 0.00409158863797309),
 ('体育馆', 99, 0.0038948776457628453)]


In [65]:

# ['nh'] 人名词
pprint(get_top(posdict['nh'], num=25))


[('•', 79, 0.04020356234096692),
 ('邱德拔', 75, 0.03816793893129771),
 ('黄', 62, 0.03155216284987277),
 ('小明', 51, 0.025954198473282442),
 ('张华', 43, 0.02188295165394402),
 ('😄', 36, 0.0183206106870229),
 ('丁满', 27, 0.013740458015267175),
 ('🐻', 27, 0.013740458015267175),
 ('周其仁', 24, 0.012213740458015267),
 ('苗苗', 21, 0.010687022900763359),
 ('ˊ_>ˋ', 20, 0.010178117048346057),
 ('东哥', 19, 0.009669211195928753),
 ('李萍', 15, 0.007633587786259542),
 ('讲堂', 14, 0.0071246819338422395),
 ('熊哥', 13, 0.0066157760814249365),
 ('夏业良', 12, 0.0061068702290076335),
 ('陈独秀', 12, 0.0061068702290076335),
 ('〜', 12, 0.0061068702290076335),
 ('张', 12, 0.0061068702290076335),
 ('姚贝娜', 12, 0.0061068702290076335),
 ('钱理群', 12, 0.0061068702290076335),
 ('习', 11, 0.0055979643765903305),
 ('π', 9, 0.004580152671755725),
 ('元培', 9, 0.004580152671755725),
 ('丁', 9, 0.004580152671755725)]


In [66]:

# ['ni'] 组织名
pprint(get_top(posdict['ni'], num=25))


[('北京大学', 5705, 0.9824349922507318),
 ('清华大学', 69, 0.011882211124504908),
 ('教育部', 9, 0.0015498536249354228),
 ('中山大学', 8, 0.0013776476666092646),
 ('新华社', 5, 0.0008610297916307905),
 ('司法部', 2, 0.00034441191665231615),
 ('公安部', 2, 0.00034441191665231615),
 ('国务院', 2, 0.00034441191665231615),
 ('老弱病残孕联队', 1, 0.00017220595832615808),
 ('华大学', 1, 0.00017220595832615808),
 ('外交部', 1, 0.00017220595832615808),
 ('卫生部', 1, 0.00017220595832615808),
 ('北京大学生', 1, 0.00017220595832615808)]


In [67]:

# ['nz'] other proper name
pprint(get_top(posdict['nz'], num=25))


[('康博思', 127, 0.2545090180360721),
 ('新华', 29, 0.05811623246492986),
 ('光华', 26, 0.052104208416833664),
 ('英语', 17, 0.03406813627254509),
 ('复旦', 13, 0.026052104208416832),
 ('中文', 11, 0.022044088176352707),
 ('可乐', 11, 0.022044088176352707),
 ('北京大学生', 10, 0.02004008016032064),
 ('新浪', 9, 0.018036072144288578),
 ('日语', 9, 0.018036072144288578),
 ('维基百科', 8, 0.01603206412825651),
 ('同济', 7, 0.014028056112224449),
 ('富士康', 7, 0.014028056112224449),
 ('雄安', 7, 0.014028056112224449),
 ('微纳', 7, 0.014028056112224449),
 ('中华', 5, 0.01002004008016032),
 ('腾讯', 5, 0.01002004008016032),
 ('赛克勒', 5, 0.01002004008016032),
 ('正大', 4, 0.008016032064128256),
 ('协和', 4, 0.008016032064128256),
 ('百度', 4, 0.008016032064128256),
 ('英皇', 3, 0.006012024048096192),
 ('九区杯', 3, 0.006012024048096192),
 ('托福', 3, 0.006012024048096192),
 ('英文', 3, 0.006012024048096192)]


In [68]:

# ['v'] verb len >= 2
pprint(get_top([x for x in posdict['v'] if len(x[0])>=2], num=25))


[('吃饭', 195, 0.015396762731938412),
 ('可以', 153, 0.012080536912751677),
 ('没有', 135, 0.010659297275957363),
 ('理教', 134, 0.010580339518357679),
 ('发现', 130, 0.010264508487958943),
 ('看到', 121, 0.009553888669561784),
 ('回来', 118, 0.009317015396762732),
 ('觉得', 118, 0.009317015396762732),
 ('工作', 116, 0.009159099881563364),
 ('知道', 104, 0.008211606790367153),
 ('还有', 98, 0.007737860244769049),
 ('毕业', 93, 0.007343071456770628),
 ('纪念', 90, 0.007106198183971575),
 ('开始', 87, 0.006869324911172522),
 ('起来', 81, 0.006395578365574418),
 ('好像', 73, 0.005763916304776944),
 ('研究', 69, 0.005448085274378208),
 ('出来', 69, 0.005448085274378208),
 ('发展', 67, 0.0052901697591788395),
 ('应该', 64, 0.005053296486379787),
 ('教育', 63, 0.004974338728780102),
 ('继续', 61, 0.004816423213580734),
 ('签到', 60, 0.00473746545598105),
 ('表示', 56, 0.004421634425582313),
 ('考上', 55, 0.0043426766679826295)]


In [69]:

# foreign words	
pprint(get_top([x for x in posdict['ws'] if len(x[0])>=4], num=25))


[('sapi', 22, 0.002237363978439947),
 ('brama', 19, 0.0019322688904708635),
 ('high', 15, 0.0015254754398454184),
 ('nite', 15, 0.0015254754398454184),
 ('PM2.5', 11, 0.0011186819892199736),
 ('RESIM', 11, 0.0011186819892199736),
 ('bkkl', 11, 0.0011186819892199736),
 ('SoftOblivion', 11, 0.0011186819892199736),
 ('dodo', 9, 0.0009152852639072511),
 ('sigh', 8, 0.0008135869012508898),
 ('Instagram', 8, 0.0008135869012508898),
 ('wifi', 6, 0.0006101901759381674),
 ('University', 6, 0.0006101901759381674),
 ('Brama', 5, 0.0005084918132818062),
 ('http', 5, 0.0005084918132818062),
 ('zadbee', 5, 0.0005084918132818062),
 ('B535', 5, 0.0005084918132818062),
 ('with', 5, 0.0005084918132818062),
 ('Soft', 5, 0.0005084918132818062),
 ('BIOPIC', 5, 0.0005084918132818062),
 ('wisteria', 4, 0.0004067934506254449),
 ('done', 4, 0.0004067934506254449),
 ('iPad', 4, 0.0004067934506254449),
 ('lite', 4, 0.0004067934506254449),
 ('bkkls', 4, 0.0004067934506254449)]


In [70]:
# 命名实体
print(nedict.keys())


dict_keys(['O', 'S-Ns', 'S-Nh', 'B-Ni', 'I-Ni', 'E-Ni', 'S-Ni', 'B-Ns', 'E-Ns', 'B-Nh', 'E-Nh', 'I-Ns', 'I-Nh'])


In [71]:


# 人名
pprint(get_top(nedict['S-Nh'], num=25))


[('•', 79, 0.043574186431329286),
 ('邱德拔', 73, 0.04026475455046884),
 ('黄', 61, 0.03364589078874793),
 ('小明', 49, 0.02702702702702703),
 ('张华', 43, 0.023717595146166576),
 ('😄', 36, 0.019856591285162713),
 ('丁满', 27, 0.014892443463872035),
 ('🐻', 27, 0.014892443463872035),
 ('周其仁', 24, 0.013237727523441808),
 ('苗苗', 20, 0.011031439602868174),
 ('ˊ_>ˋ', 19, 0.010479867622724766),
 ('东哥', 19, 0.010479867622724766),
 ('李萍', 15, 0.00827357970215113),
 ('讲堂', 14, 0.007722007722007722),
 ('熊哥', 13, 0.007170435741864313),
 ('夏业良', 12, 0.006618863761720904),
 ('陈独秀', 12, 0.006618863761720904),
 ('〜', 12, 0.006618863761720904),
 ('钱理群', 12, 0.006618863761720904),
 ('张', 11, 0.006067291781577496),
 ('姚贝娜', 11, 0.006067291781577496),
 ('π', 9, 0.004964147821290678),
 ('丁', 9, 0.004964147821290678),
 ('嗷', 8, 0.00441257584114727),
 ('胡适', 8, 0.00441257584114727)]


In [72]:

# 机构名
pprint(get_top(nedict['S-Ni'], num=25))


[('北京大学', 4217, 0.9791037845368006),
 ('清华大学', 60, 0.013930810308799628),
 ('中山大学', 7, 0.00162526120269329),
 ('教育部', 6, 0.0013930810308799629),
 ('新华社', 5, 0.0011609008590666356),
 ('中科院', 4, 0.0009287206872533086),
 ('司法部', 2, 0.0004643603436266543),
 ('公安部', 2, 0.0004643603436266543),
 ('老弱病残孕联队', 1, 0.00023218017181332715),
 ('外交部', 1, 0.00023218017181332715),
 ('国务院', 1, 0.00023218017181332715),
 ('北京大学生', 1, 0.00023218017181332715)]


In [73]:

# 地名
pprint(get_top(nedict['S-Ns'], num=25))

[('中国', 216, 0.16450875856816452),
 ('北京', 126, 0.09596344249809596),
 ('东南门', 53, 0.04036557501904037),
 ('东门', 53, 0.04036557501904037),
 ('五四路', 46, 0.035034272658035034),
 ('西南门', 32, 0.024371667936024372),
 ('燕南', 25, 0.01904036557501904),
 ('燕南园', 24, 0.01827875095201828),
 ('美国', 22, 0.016755521706016754),
 ('深圳', 22, 0.016755521706016754),
 ('燕南路', 21, 0.015993907083015995),
 ('上海', 20, 0.015232292460015232),
 ('燕园', 16, 0.012185833968012186),
 ('畅春园', 16, 0.012185833968012186),
 ('香港', 14, 0.010662604722010662),
 ('浙江', 12, 0.00913937547600914),
 ('中东', 11, 0.008377760853008377),
 ('日本', 11, 0.008377760853008377),
 ('雄安', 11, 0.008377760853008377),
 ('西直门', 10, 0.007616146230007616),
 ('北京市', 9, 0.006854531607006854),
 ('物美北', 8, 0.006092916984006093),
 ('英国', 8, 0.006092916984006093),
 ('亚洲', 8, 0.006092916984006093),
 ('美', 8, 0.006092916984006093)]


In [100]:
# 以上内容均为自动生成...